In [ ]:
!pip install transformers faiss-cpu openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 77.1 MB/s eta 0:00:00


In [ ]:
# -*- coding: utf-8 -*-
import os, json, hashlib
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModel
import torch
from openai import OpenAI

# ✅ Initialize OpenAI client (use API key directly or environment variable)
client = OpenAI(api_key="")  # <- Replace with your GPT-4o API Key

# ✅ Load document chunks
with open("output.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

# ✅ Compute MD5 for cache control
def compute_md5(file_path):
    with open(file_path, "rb") as f:
        return hashlib.md5(f.read()).hexdigest()

json_md5 = compute_md5("output.json")

# ✅ Load embedding model (bge-large-en-v1.5)
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-large-en-v1.5")
model = AutoModel.from_pretrained("BAAI/bge-large-en-v1.5")
model.eval()

# ✅ Convert text to embedding
def get_embedding(text):
    encoded_input = tokenizer("query: " + text, return_tensors='pt', max_length=512, truncation=True)
    with torch.no_grad():
        model_output = model(**encoded_input)
    token_embeddings = model_output.last_hidden_state
    attention_mask = encoded_input['attention_mask']
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, dim=1)
    sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
    mean_pooled = sum_embeddings / sum_mask
    return mean_pooled.squeeze().numpy()

# ✅ Embed document chunks in batch
def embed_chunks(chunks):
    return np.array([get_embedding(c["text"]) for c in chunks])

# ✅ Caching mechanism
if os.path.exists("vectors.npy") and os.path.exists("vectors.md5"):
    with open("vectors.md5", "r") as f:
        saved_md5 = f.read().strip()
    if saved_md5 == json_md5:
        print("✅ Loaded cached vectors.")
        vectors = np.load("vectors.npy")
    else:
        print("🔄 Manual updated. Recomputing vectors...")
        vectors = embed_chunks(chunks)
        np.save("vectors.npy", vectors)
        with open("vectors.md5", "w") as f:
            f.write(json_md5)
else:
    print("🔄 No cached vectors found. Computing now...")
    vectors = embed_chunks(chunks)
    np.save("vectors.npy", vectors)
    with open("vectors.md5", "w") as f:
        f.write(json_md5)

# ✅ Build FAISS index
def build_faiss_index(vectors):
    dim = vectors.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(vectors)
    return index

index = build_faiss_index(vectors)

# ✅ Search and return top_k results with similarity score
def search(query, index, chunks, top_k=8):
    query_vector = get_embedding(query)
    D, I = index.search(np.array([query_vector]), top_k)
    results = []
    for i, idx in enumerate(I[0]):
        chunk = chunks[idx]
        chunk["similarity_score"] = float(D[0][i])
        results.append(chunk)
    return results

# Call GPT-4o
def ask_gpt4o(context_texts, query):
    # Join context content
    context = "\n\n".join([
        f"Section: {c['metadata'].get('headings', 'Unknown')}\n\n{c['text']}" for c in context_texts
    ])

    # New prompt: clearer structure and smarter instructions
    prompt = f"""
You are a clinical decision support assistant. Use only the content from the provided CANMAT clinical manual to answer the user's question.

Instructions:
- Carefully review the retrieved text and find any relevant medication recommendations, treatment considerations, or patient-specific factors.
- If multiple options exist, summarize the top 1–2 and explain briefly why they are preferred.
- Reference the section title or context when applicable.
- If there is no explicit answer, you may reasonably infer from related sections (e.g. similar symptoms, comorbidities, or past treatments), but make that clear.
- If absolutely no relevant information is available in the manual, respond with: "No clear recommendation found in the manual."

Please answer in 3–4 sentences using clear and clinically relevant language.

---
Clinical Manual:
{context}

Question:
{query}

Answer:
"""

    # Call GPT-4o API
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": "You are a medical assistant trained to follow evidence-based clinical guidelines only."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.2,
        max_tokens=512
    )

    return response.choices[0].message.content.strip()

# ✅ Test question list
queries = [
    "I have a patient who can’t take lithium but is currently acutely manic. Which medication should I try next?",
    "My patient with BDI-Depression had side effects from quetiapine. Which medication should I try next?",
    "I have a patient experiencing hypomania and symptoms of depression. What is the recommended treatment?",
    "My patient with Bipolar II disorder is acutely depressed. What medication should I try?",
    "My patient with BDII will not adhere to any treatment plan that causes weight gain. How should I treat his depression?",
    "My patient responded well to divalproex for acute mania in the past, but is now pregnant. How should I treat her?",
    "My patient with Type 1 Bipolar and comorbid diabetes is experiencing depression. What is the recommended treatment?",
    "My patient has hepatic dysfunction and is hypomanic. What is the recommended treatment?",
    "My patient with Type 1 Bipolar and cardiovascular disease is experiencing depression. What is the recommended treatment?",
    "I have a Bipolar Type II patient with renal disease who is experiencing depression. What is the recommended treatment?",
    "My patient is experiencing their fourth episode of mania this year. How should I proceed?",
    "I have a patient who is manic and experiencing psychotic features. I am considering the possibility of schizoaffective disorder. How should I proceed?",
    "My patient is currently acutely depressed. He takes quetiapine for anti-manic prophylaxis, but it doesn't work for his depression. What should I try next?",
    "My patient needs a quick treatment for their depression as they have pronounced suicidality. What should we use?",
    "My patient with bipolar II responded well to lamotrigine in the past for acute depression. However, he was just admitted due to significant active suicidal ideation which I would like to treat as soon as possible. What should we use?",
    "My patient is experiencing acute depression with anxious distress. Which first-line treatment should I choose?",
    "I treated my patient's acute mania with lithium. How should I proceed with maintenance treatment?",
    "I treated my patient's acute depression with quetiapine. How should I proceed with maintenance treatment?",
    "What is the recommended maintenance treatment for my patient with BDII, if depression was treated with quetiapine?",
    "I'm about to start treating a patient with lithium, what are common side effects at the start?"
]

# ✅ Main loop: retrieval + print chunks + GPT call
for i, q in enumerate(queries, 1):
    retrieved_chunks = search(q, index, chunks)
    print(f"\n🟦 QUESTION {i}: {q}")
    #print(f"🔎 Retrieved Chunks:")
    for j, c in enumerate(retrieved_chunks, 1):
        title = c["metadata"].get("headings", "No heading")
        preview = c["text"][:300].replace("\n", " ")
        score = round(c.get("similarity_score", 0.0), 2)
        #print(f"-- Chunk {j} -- [Section: {title}] [Score: {score}]\n{preview}...\n")

    answer = ask_gpt4o(retrieved_chunks, q)
    print(f"\n🤖 GPT-4o Answer:\n{answer}")


✅ Loaded cached vectors.

🟦 QUESTION 1: I have a patient who can’t take lithium but is currently acutely manic. Which medication should I try next?

🤖 GPT-4o Answer:
No clear recommendation found in the manual.

🟦 QUESTION 2: My patient with BDI-Depression had side effects from quetiapine. Which medication should I try next?

🤖 GPT-4o Answer:
For a patient with BDI-Depression who experienced side effects from quetiapine, considering an alternative serotonin-dopamine activity modulator with a more favorable side effect profile is advisable. Aripiprazole and brexpiprazole are recommended as first-line agents due to their efficacy and tolerability, making them suitable options to try next. These medications have a lower risk of sedation and metabolic side effects compared to quetiapine, which may improve the patient's overall treatment experience.

🟦 QUESTION 3: I have a patient experiencing hypomania and symptoms of depression. What is the recommended treatment?

🤖 GPT-4o Answer:
For a p